In [ ]:
# example to work with MPC.
%load_ext autoreload
%autoreload 2


import crocoddyl
import time

import FlexivPy.controllers.mpc as mpc

import pinocchio as pin
import numpy as np
from FlexivPy.robot.model.pinocchio import FlexivModel
from FlexivPy.sim.MuJoCo import FlexivSimMujoco as FlexivSim
from FlexivPy.robot.dds.flexiv_messages import FlexivCmd


urdf = None  # lets use the default
# NOTE: to use the robot with the gipper and camera you should use this urdf!
# urdf = "/home/FlexivPy/FlexivPy/assets/real_rizon_with_gripper.urdf"
flexiv_model = FlexivModel(urdf=urdf, render=True)
flexiv_model.display(flexiv_model.q0)

In [ ]:
# example of running MPC

q0 = flexiv_model.q0
D = flexiv_model.getInfo(q0, np.zeros(7))
T0 = D["poses"]["link7"]


# we want
dx_des = [0, 0.5, 0.0]
Tdes = np.copy(T0)
Tdes[:3, 3] += np.array(dx_des)

mpc_cfg = mpc.Mpc_cfg()


mpc_generator = mpc.Mpc_generator(
    flexiv_model.robot.model,
    x0=np.concatenate([flexiv_model.q0, np.zeros(7)]),
    Tdes=Tdes,
    mpc_cfg=mpc_cfg,
)

mpc_generator.solver.solve()


u = mpc_generator.solver.us[0]
x_des = mpc_generator.solver.xs[1]

print("sending to the robot!")

print("u", u)
print("x_des", x_des)


display = crocoddyl.MeshcatDisplay(flexiv_model.robot)

display.rate = -1
display.freq = 1
print("displaying")
for i in range(2):
    display.displayFromSolver(mpc_generator.solver)
    time.sleep(1.0)

In [ ]:
model = FlexivModel()
control_mode = "velocity"
control_mode = "torque"
robot = FlexivSim(render=True, dt=0.002, mode=control_mode)
# Direct simultion, using the position interface.
robot.reset_state_robot(model.q0, np.zeros_like(model.q0))

In [ ]:
def get_error(x, Tdes, flexiv_model):
    Tcurrent = flexiv_model.getInfo(x[:7], np.zeros(7))["poses"]["link7"]
    return np.linalg.norm(pin.log6(np.linalg.inv(Tdes) @ Tcurrent))


display = crocoddyl.MeshcatDisplay(flexiv_model.robot)


cmd = FlexivCmd()
cmd.q = model.q0
robot.set_cmd(cmd)
robot.viewer.sync()
s = robot.get_robot_state()
T_cmd = model.getInfo(model.q0, np.zeros(7))["poses"]["link7"].copy()
T_cmd[1, 3] += 0.5


mpc_cfg = mpc.Mpc_cfg(
    gripperPose_run=100,
    gripperPose_terminal=100,
)


mpc_generator = mpc.Mpc_generator(
    flexiv_model.robot.model,
    np.concatenate([flexiv_model.q0, np.zeros(7)]),
    T_cmd,
    mpc_cfg,
)


# num steps
num_sim_steps = int(mpc_generator.mpc_cfg.dt / robot.dt)  # lets say we solve the

s = robot.get_robot_state()


q_des = s.q

alpha = 0.9  # we can smooth the velocity cmd.

display = crocoddyl.MeshcatDisplay(flexiv_model.robot)

display.rate = -1
display.freq = 1

for i in range(5000):
    s = robot.get_robot_state()
    mpc_generator.update_x0(np.concatenate([s.q, s.dq]))
    mpc_generator.solver.solve()

    if i % 10 == 0:
        time.sleep(0.01)  # wait because print in callback of solver is async
        print(
            f"error of last state of MPC is { get_error(mpc_generator.solver.xs[-1][:7], T_cmd, flexiv_model)  }"
        )
        print(f"error of state is {get_error(s.q, T_cmd, flexiv_model)}")
        display.displayFromSolver(mpc_generator.solver)

    if control_mode == "velocity":

        dq_des = (
            mpc_generator.solver.xs[1][:7] - mpc_generator.solver.xs[0][:7]
        ) / mpc_generator.mpc_cfg.dt

        print("dq_des", dq_des)
        cmd.dq = s.dq * (1 - alpha) + dq_des * alpha
    if control_mode == "torque":
        cmd.tau_ff = mpc_generator.solver.us[0]
        cmd.kv = 0.1 * np.ones(7)  # small - kv * dq
        cmd.dq = np.zeros(7)
        cmd.tau_ff_with_gravity = True

    robot.set_cmd(cmd)
    tic = time.time()
    for _ in range(num_sim_steps):
        robot.step()
    toc = time.time()
    time.sleep(max(num_sim_steps * robot.dt - (toc - tic), 0))

In [ ]:
# now we can run an MPC controller!